In [42]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromi

In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.firefox.options import Options
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time
import functools
import re
import requests

In [44]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [0]:
def clean_stats_frame(df,teams,player_id, team_ind ,player_name, columns, drop_indexes, drop_cols, id_vars, tm_id): 
    df.drop(df.tail(1).index,inplace=True)
    df.iloc[:,team_ind] = teams
    df = df.drop(df.columns[drop_indexes], axis = 1)
    df.columns = columns
    if not(drop_cols == []):
        df = df.drop(drop_cols, axis = 1)
    df.insert(0,"tm_Id", tm_id)
    df.insert(1,"Player_Id", player_id)
    df.insert(2,"Name", player_name)
    df_long = pd.melt(df, id_vars = id_vars, var_name = "Attribute")
    return df_long

In [0]:
def check_if_exists(dest):
    result = True
    try:
        open(dest)
    except IOError:
        result = False
    return result

In [0]:
def create_empty_df(file_dest, columns):
    data_frame = pd.DataFrame(columns=columns)
    data_frame.to_csv(file_dest)


def create_csv_dfs():
    create_empty_df("./Scrapped_Data/injuries.csv", columns = ["sofifa_id","Name","Reason","Start Date","End Date"])
    create_empty_df("./Scrapped_Data/trophies.csv", columns = ["sofifa_id","Name","Competition","Trophy","Season"])
    create_empty_df("./Scrapped_Data/markval.csv", columns = ['Name', 'Club', 'League', 'Season', 'Market Value', "tm_Id"])

In [0]:
def sign_in(driver):
    driver.get("https://www.instagram.com/")
    username = "grno53"
    password = "grnogrno22"
    time.sleep(10)
    username_field = driver.find_elements_by_css_selector(".zyHYP")[0]
    pass_field = driver.find_elements_by_css_selector(".zyHYP")[1]
    username_field.send_keys(username)
    pass_field.send_keys(password)
    sign_in_button = driver.find_elements_by_css_selector(".y3zKF , .y3zKF ._4EzTm")[0]
    sign_in_button.click()
    time.sleep(10)

In [0]:
class Player:
    def __init__(self, id, link, driver, df_path, stats_path, nat_stats_path, transfers_path, name):
        # self.name = "no_info"
        self.driver = driver
        self.id = id
        self.link = link
        self.info_df = None
        self.followers = None
        self.df_path = df_path
        self.stats_path = stats_path
        self.nat_stats_path = nat_stats_path
        self.transfers_path = transfers_path
        self.position = None
        self.tm_Id = None
        self.name = name
        print(self.id)
        self.get_tm_id()
        self.get_info()
        self.get_followers()
        self.data_to_append()
        self.get_player_stats()
        self.get_nat_stats()
        self.get_transfers()

    def get_transfers(self):
        print("trying to get player's transfers")
        transfers = pd.DataFrame(columns=['Player_Id', "tm_Id", 'Name', 'From', 'To', 'Fee', 'Market Value', 'Season', 'Date'])
        try:
            link = self.link.replace("profil","transfers")
            self.driver.get(link)
            bs_obj = BeautifulSoup(self.driver.page_source, 'html.parser')
            rows = bs_obj.find_all('table')[0].find('tbody').find_all('tr',{"class":"zeile-transfer"})
            for row in rows:
                cols = row.find_all('td')
                data = {
                    "Player_Id": self.id,
                    "tm_Id": self.tm_Id,
                    "Name": self.name,
                    "From": cols[5].get_text(),
                    "To": cols[9].get_text(),
                    "Fee": cols[11].get_text(),
                    "Market Value": cols[10].get_text(),
                    "Season": cols[0].get_text(),
                    "Date": cols[1].get_text()
                }
                transfers = transfers.append(data, ignore_index=True)
                transfers.set_index("tm_Id", inplace=True, drop=False)
                transfers.to_csv(self.transfers_path, mode = "a", header= False)
        except Exception as e:
            print(str(e))
        # self.transfers_df = transfers

    def go_detailed_page(self):
        detailed = self.driver.find_elements_by_css_selector(".kartei-button-body")[1]
        self.driver.execute_script("arguments[0].click();", detailed)
        time.sleep(5)

    def get_tm_id(self):
        digits = re.findall(r"\d", self.link)
        self.tm_Id = functools.reduce(lambda a,b : a+b,digits)

    def get_info(self):
        print("trying to get information about the player")
        try:
            self.driver.get(self.link)
            bs_obj = BeautifulSoup(self.driver.page_source, 'html.parser')
            # name = bs_obj.find_all('h1')[0].get_text()
            table = bs_obj.find_all('table')[0]
            df = pd.read_html(str(table))[0]
            df.columns = ["Key", "Value"]
            self.info_df = df
            # self.name = name
        except Exception as e:
            print(str(e))

    def get_followers(self):
        print("trying to get information about the player's followers")
        insta_xpath = "//*[@title='Instagram']"
        insta_link = None
        followers = None
        try:
            insta_link = self.driver.find_element_by_xpath(insta_xpath).get_attribute("href")
            if not(insta_link is None):
                pat = r'.com?\/(.*)/.*'
                username = re.findall(pat, insta_link)[0]
                def_url = "https://www.instagram.com/<username>/?__a=1"
                url = def_url.replace("<username>", username)
                r = requests.get(url = url) 
                data = r.json()
                followers = data["graphql"]["user"]["edge_followed_by"]["count"]
        except Exception as e:
            print("no info about followers, trying to visit the page")
            print(str(e))
            try:
                insta_link = self.driver.find_element_by_xpath(insta_xpath).get_attribute("href")
                if not(insta_link is None):
                    self.driver.get(insta_link)
                    time.sleep(2)
                    bs_obj = BeautifulSoup(self.driver.page_source, 'html.parser')
                    followers = bs_obj.find_all("span", class_="g47SY")[1]["title"]
            except Exception as e:
                print("another exception, catching it")
                print(str(e))
        finally:
            if(followers is None):
                print("nothing found")
                followers = "no info"
            else:
                print("number of followers",followers)
        self.followers = followers

    def extract_cell(self, key):
        result = None
        df = self.info_df
        try:
            result = df[df["Key"] == key]["Value"].values[0]
        except:
            print("the player does not have " + str(key))
            result = "no_info"
        return result

    def data_to_append(self):
        try:
            data = {
                'Id': self.id,
                'tm_Id': self.tm_Id,
                'Name':  self.name,
                'Team':  self.extract_cell("Current club:"),
                'Nationality':  self.extract_cell("Citizenship:"),
                'Date of Birth':  self.extract_cell("Date of birth:"),
                'Height':  self.extract_cell("Height:"),
                'Strong Foot':  self.extract_cell("Foot:"),
                'Position':  self.extract_cell("Position:"),
                'Joined':  self.extract_cell("Joined:"),
                'Contract Expires':  self.extract_cell("Contract expires:"),
                'Followers':  self.followers
            }
            df = pd.DataFrame(columns = ['Id','tm_Id','Name', 'Team', 'Nationality', 'Date of Birth', 'Height', 'Strong Foot', 'Position', 'Joined', 'Contract Expires', 'Followers'])
            df = df.append(data, ignore_index = True)
            df.set_index("tm_Id", inplace=True, drop=False)
            df.to_csv(self.df_path, mode = "a", header= False)
            self.position = data["Position"]
            self.data = data
        except Exception as e:
            print(str(e))
        # self.position = data["Position"]
        # self.data = data
        # self.df = df

    def get_player_stats(self):
        print("trying to get the player's statistics")
        df_long = None
        teams_xpath = "//*[contains(concat( ' ', @class, ' ' ), concat( ' ', 'zentriert', ' ' ))]//img"
        # position_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "dataValue", " " ))]'
        try:
            stats_link = self.link.replace("profil","leistungsdatendetails")
            self.driver.get(stats_link)
            self.go_detailed_page()
            get_name = lambda input: input.get_attribute('alt')
            teams = self.driver.find_elements_by_xpath(teams_xpath)
            teams = list(map(get_name, teams))
            bs_obj = BeautifulSoup(self.driver.page_source, 'html.parser')
            table = bs_obj.find_all('table')[1]
            df = pd.read_html(str(table))[0]
            position = self.extract_cell("Position:")
            if(position == "Goalkeeper"):
                columns = ["Season","Team","Competition","Squad",
                    "Appearances","PPG","Goals","Own goals","Substitutions on",
                    "Substitutions off","Yellow Cards","Second yellow cards","Red cards",
                    "Goals conceded","Clean sheets","Minutes played"]
                drop_indexes = [3,17]
                drop_cols = ["Goals"]
            else:
                columns = ["Season","Team","Competition","Squad",
                "Appearances","PPG","Goals","Assists","Own goals","Substitutions on",
                "Substitutions off","Yellow Cards","Second yellow cards","Red cards",
                "Penalty goals","Minutes per goal","Minutes played"]
                drop_indexes = [3,18]
                drop_cols = []
            id_vars = ["Player_Id","tm_Id","Name","Team","Season","Competition"]
            df_long = clean_stats_frame(df = df, teams = teams, team_ind = 1, player_id = self.id,
            player_name = self.name, columns = columns, drop_indexes = drop_indexes,
            drop_cols = drop_cols, id_vars = id_vars, tm_id = self.tm_Id)
            df_long.set_index("tm_Id", inplace=True, drop=False)
            df_long.to_csv(self.stats_path, mode = "a", header= False)
        except Exception as e:
            print(str(e))

    def get_nat_stats(self):
        print("trying to get the player's national team statistics")
        nat_team_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "tooltipstered", " " ))]'
        df_long = None
        try:
            nat_link = self.link.replace("profil","nationalmannschaft")
            self.driver.get(nat_link)
            self.go_detailed_page()
            nat_team = self.driver.find_element_by_xpath(nat_team_xpath).text
            bs_obj = BeautifulSoup(self.driver.page_source, 'html.parser')
            table = bs_obj.find_all('table')[2]
            df = pd.read_html(str(table))[0]
            position = self.extract_cell("Position:")
            if(position == "Goalkeeper"):
                columns = ["National Team","Competition",
                        "Appearances","Goals","Own goals","Substitutions on",
                        "Substitutions off","Yellow Cards","Second yellow cards","Red cards",
                        "Goals conceded","Clean sheets","Minutes played"]
                drop_indexes = [13]
                drop_cols = ["Goals"]
            else:
                drop_indexes = [14]
                columns = ["National Team","Competition","Squad",
                        "Goals","Assists","Own goals","Substitutions on",
                        "Substitutions off","Yellow Cards","Second yellow cards","Red cards",
                        "Penalty goals","Minutes per goal","Minutes played"]
                drop_cols = []
            id_vars = ["Player_Id","tm_Id","Name","National Team","Competition"]
            df_long = clean_stats_frame(df = df, teams = nat_team, team_ind = 0, player_id = self.id,
            player_name = self.name, columns = columns, drop_indexes = drop_indexes,
            drop_cols = drop_cols, id_vars = id_vars, tm_id = self.tm_Id)
            df_long.set_index("tm_Id", inplace=True, drop=False)
            df_long.to_csv(self.nat_stats_path, mode = "a", header= False)
        except Exception as e:
            print(str(e))


In [0]:
player_links = pd.read_csv("/content/drive/My Drive/Capstone/Prerequisite Data/playerlinks.csv")['Player_url']
player_names = pd.read_csv("/content/drive/My Drive/Capstone/Prerequisite Data/playerlinks.csv")['Player_url']
# a = pd.read_csv("/content/drive/My Drive/Capstone/Prerequisite Data/playerlinks.csv")
# a.to_csv('data.csv')
# !cp data.csv "drive/My Drive/Capstone/Scrapped_Data"

In [0]:
def start_scrapping(driver, start, end, player_links):
    dest_cols = {
        "drive/My Drive/Capstone/Scrapped_Data/Players.csv" : ['Id','tm_Id','Name', 'Team', 'Nationality', 'Date of Birth', 'Height', 'Strong Foot', 'Position', 'Joined', 'Contract Expires', 'Followers'],
        "drive/My Drive/Capstone/Scrapped_Data/stats.csv" : ['Player_Id', 'tm_Id', 'Name', 'Team', 'Season', 'Competition', 'Attribute','value'],
        "drive/My Drive/Capstone/Scrapped_Data/natstats.csv" : ["Player_Id", "tm_Id", "Name","National Team","Competition", "Attribute", "value"],
        "drive/My Drive/Capstone/Scrapped_Data/Transfers.csv" : ['Player_Id', "tm_Id", 'Name', 'From', 'To', 'Fee', 'Market Value', 'Season', 'Date']
    }
    for dest,cols  in dest_cols.items():
        if(not(check_if_exists(dest = dest))):
            create_empty_df(file_dest = dest, columns = cols)

    sign_in(driver)
    for id in range(start, end):
        player = None
        try:
            link = player_links[id]
            name = player_names[id]
            dests = list(dest_cols.keys())
            player = Player(id = id, link = link, driver = driver, df_path = dests[0], stats_path = dests[1],
            nat_stats_path = dests[2], transfers_path = dests[3], name = name)
        except Exception as e:
            print("The exception message", str(e))
            break
        del player
    driver.quit()
    print("Finished scrapping")

In [0]:
start_scrapping(driver, start = 10000, end = 10200, player_links = player_links)

10000
trying to get information about the player
trying to get information about the player's followers
no info about followers, trying to visit the page
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title='Instagram']"}
  (Session info: headless chrome=80.0.3987.87)

another exception, catching it
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title='Instagram']"}
  (Session info: headless chrome=80.0.3987.87)

nothing found
the player does not have Height:
the player does not have Foot:
trying to get the player's statistics
trying to get the player's national team statistics
list index out of range
trying to get player's transfers
10001
trying to get information about the player
trying to get information about the player's followers
no info about followers, trying to visit the page
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title='Instagram']"}
  (Session info: